# 数据集处理与模型训练

本示例展示如何将万卷丝绸越南语数据集转换为 LlamaFactory 训练所需的指令格式。

## 任务说明

### 输入数据格式（原始 JSONL）
```json
{
  "type": "culture",
  "prompt": "Văn hóa dùng đũa của người Việt Nam?",
  "completion": "Đũa là một đồ vật...",
  "labels": {"unsafe_word_min_level": "NF"},
  "track_id": "1",
  "language": "vi",
  "domain": "human-annotation"
}
```

### 输出数据格式（指令格式）
```json
{
  "instruction": "Văn hóa dùng đũa của người Việt Nam?",
  "input": "",
  "output": "Đũa là một đồ vật..."
}
```

## 处理流程

1. 加载原始 JSONL 文件
2. 提取 prompt 和 completion 字段
3. 转换为 instruction/input/output 格式
4. 按 9:1 分割训练集和验证集
5. 保存为 JSON 文件

In [ ]:
import json
import os
from pathlib import Path
from typing import List, Dict
import random

In [6]:

random.seed(42)


def load_jsonl(file_path: str) -> List[Dict]:
    """加载 jsonl 文件"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data


# 数据处理函数

## 函数说明

### load_jsonl(file_path)
加载 JSONL 格式的数据文件，逐行解析 JSON 对象。

### convert_to_instruction_format(data)
将原始数据转换为 LlamaFactory 所需的指令格式，提取 `prompt` 作为 `instruction`，`completion` 作为 `output`。

### split_train_val(data, val_ratio=0.1)
将数据随机分割为训练集和验证集，默认验证集比例为 10%。

In [7]:

def convert_to_instruction_format(data: List[Dict]) -> List[Dict]:
    """
    将原始数据转换为 instruction/input/output 格式

    原始格式: {"type": "...", "prompt": "...", "completion": "...", ...}
    转换后: {"instruction": prompt, "input": "", "output": completion}
    """
    converted = []
    for item in data:
        # 跳过无效数据
        if not item.get("prompt") or not item.get("completion"):
            continue

        # 跳过示例数据
        if item.get("prompt") == "Question" and item.get("completion") == "Answer":
            continue

        converted_item = {
            "instruction": item["prompt"],
            "input": "",
            "output": item["completion"]
        }
        converted.append(converted_item)

    return converted


In [8]:


def split_train_val(data: List[Dict], val_ratio: float = 0.1) -> tuple:
    """分割训练集和验证集"""
    random.shuffle(data)
    split_idx = int(len(data) * (1 - val_ratio))
    train_data = data[:split_idx]
    val_data = data[split_idx:]
    return train_data, val_data


In [11]:

base_dir = Path(".")
raw_data_path = base_dir / "TargetData" / "raw"/"OpenDataLab___WanJuanSiLu2O"/"raw"/"sft"/"vi" / "vi.jsonl"
output_dir = base_dir / "TargetData" / "processed" / "dialogue_format"


# 文件路径配置

## 路径说明

- **原始数据**: `TargetData/raw/.../vi.jsonl` - OpenXLab 下载的越南语数据集
- **输出目录**: `TargetData/processed/dialogue_format/` - 处理后的训练集和验证集

In [12]:

print(f"Loading data from: {raw_data_path}")
raw_data = load_jsonl(str(raw_data_path))
print(f"Loaded {len(raw_data)} samples")



Loading data from: TargetData\raw\OpenDataLab___WanJuanSiLu2O\raw\sft\vi\vi.jsonl
Loaded 23000 samples


# 数据转换执行

## 处理流程

1. 加载原始数据（约 23,000 条）
2. 过滤无效数据，转换为指令格式
3. 随机分割为训练集和验证集
4. 保存为 JSON 文件

## 预期输出

- **原始样本**: ~23,000 条
- **有效样本**: ~22,999 条
- **训练集**: ~20,699 条 (90%)
- **验证集**: ~2,300 条 (10%)

In [14]:
raw_data[0]

{'type': 'culture',
 'prompt': 'Văn hóa dùng đũa của người Việt Nam?',
 'completion': 'Đũa là một đồ vật thân thuộc dùng trong mỗi bữa ăn của người Việt Nam. Và văn hóa dùng đũa của người Việt Nam rất phong phú và mang nhiều ý nghĩa. Đũa không chỉ là công cụ ăn uống mà còn phản ánh phong cách sống và những giá trị văn hóa sâu sắc. Quy tắc dùng đũa của người VIệt Nam cũng không quá phức tạp, khi bắt đầu bữa ăn thì thường mọi người sẽ lấy đôi đũa sạch để gắp cho mọi người xung quanh, và sau đó trong bữa ăn nếu muốn gắp cho người khác họ sẽ phải đảo đầu đũa, đây là một trong những phép lịch sự cơ bản. Khi không dùng đùa nữa thì phải đặt đũa ngang ở trên bát và không nên cắm thẳng đôi đũa vào trong bát cơm vì đó được coi là điềm xấu. Ở Việt Nam còn kiêng gõ đũa vào nhau hay gõ đũa vào thành bát khi đang ăn vì họ cho rằng điều đó sẽ khiến cho ma đói tới quấy nhiều và kiêng gõ đũa là phép lịch sự mà người Việt Nam rất coi trọng.',
 'labels': {'unsafe_word_min_level': 'NF'},
 'track_id': '1',

In [13]:

print("Converting to instruction/input/output format...")
converted_data = convert_to_instruction_format(raw_data)
print(f"Converted {len(converted_data)} valid samples")


Converting to instruction/input/output format...
Converted 22999 valid samples


In [15]:
converted_data[0]

{'instruction': 'Văn hóa dùng đũa của người Việt Nam?',
 'input': '',
 'output': 'Đũa là một đồ vật thân thuộc dùng trong mỗi bữa ăn của người Việt Nam. Và văn hóa dùng đũa của người Việt Nam rất phong phú và mang nhiều ý nghĩa. Đũa không chỉ là công cụ ăn uống mà còn phản ánh phong cách sống và những giá trị văn hóa sâu sắc. Quy tắc dùng đũa của người VIệt Nam cũng không quá phức tạp, khi bắt đầu bữa ăn thì thường mọi người sẽ lấy đôi đũa sạch để gắp cho mọi người xung quanh, và sau đó trong bữa ăn nếu muốn gắp cho người khác họ sẽ phải đảo đầu đũa, đây là một trong những phép lịch sự cơ bản. Khi không dùng đùa nữa thì phải đặt đũa ngang ở trên bát và không nên cắm thẳng đôi đũa vào trong bát cơm vì đó được coi là điềm xấu. Ở Việt Nam còn kiêng gõ đũa vào nhau hay gõ đũa vào thành bát khi đang ăn vì họ cho rằng điều đó sẽ khiến cho ma đói tới quấy nhiều và kiêng gõ đũa là phép lịch sự mà người Việt Nam rất coi trọng.'}

In [16]:

# 分割训练集和验证集
print("Splitting into train/val sets...")
train_data, val_data = split_train_val(converted_data, val_ratio=0.1)
print(f"Train: {len(train_data)}, Val: {len(val_data)}")


Splitting into train/val sets...
Train: 20699, Val: 2300


In [18]:
def save_json(data: List[Dict], file_path: str) -> None:
    """保存为 JSON 文件"""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


In [19]:

# 保存文件
train_path = output_dir / "train.json"
val_path = output_dir / "val.json"

print(f"Saving train data to: {train_path}")
save_json(train_data, str(train_path))

print(f"Saving val data to: {val_path}")
save_json(val_data, str(val_path))

# 显示几个样本
print("\n=== Sample data ===")
for i, item in enumerate(train_data[:3]):
    print(f"\nSample {i+1}:")
    print(f"Instruction: {item['instruction'][:100]}...")
    print(f"Input: {item['input']}")
    print(f"Output: {item['output'][:100]}...")

print("\n=== Conversion complete ===")
print(f"Train: {train_path}")
print(f"Val: {val_path}")

Saving train data to: TargetData\processed\dialogue_format\train.json
Saving val data to: TargetData\processed\dialogue_format\val.json

=== Sample data ===

Sample 1:
Instruction: Nguồn gốc tục thờ cúng tổ tiên của người Việt?...
Input: 
Output: Tục thờ cúng tổ tiên có nguồn gốc từ nền kinh tế nông nghiệp trong xã hội phụ quyền xưa. Khi Nho giá...

Sample 2:
Instruction: Vui lòng tạo lệnh BASH dựa trên đầu vào được cung cấp để hoàn thành nhiệm vụ mong muốn.
Lệnh ngôn ng...
Input: 
Output: find -name \*.txt...

Sample 3:
Instruction: Viết mô tả về chuyến du lịch khinh khí cầu. ...
Input: 
Output: Một chuyến du lịch khinh khí cầu là một cách thú vị để chụp phong cảnh từ một góc nhìn khác. Hãy sẵn...

=== Conversion complete ===
Train: TargetData\processed\dialogue_format\train.json
Val: TargetData\processed\dialogue_format\val.json


![alt text](Model/lora-train_2026-01-07-16-29-48/training_loss.png)